## Haystack custom components

We can define a custom component in Haystack as follows

```python
from haystack import component

@component
class WelcomeTextGenerator:
  """
  A component generating personal welcome message and making it upper case
  """
  @component.output_types(welcome_text=str, note=str)
  def run(self, name:str):
    return {"welcome_text": ('Hello {name}, welcome to Haystack!'.format(name=name)).upper(), 
            "note": "welcome message is ready"}
```

## Haystack pipelines with out of the box components



```python

from haystack import Document, Pipeline
from haystack.utils import Secret
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.converters import MarkdownToDocument
from haystack.components.embedders import OpenAIDocumentEmbedder
from haystack.components.preprocessors import DocumentCleaner
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.writers import DocumentWriter
from pathlib import Path
from haystack.document_stores.types import DuplicatePolicy


document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

# Initialize components
html_to_document = HTMLToDocument()
document_cleaner = DocumentCleaner(
                    remove_empty_lines=True,
                    remove_extra_whitespaces=True,
                    remove_repeated_substrings=False
                )
document_splitter = DocumentSplitter(split_by="word", split_length=5)
document_writer = DocumentWriter(document_store=document_store,
                                 policy = DuplicatePolicy.OVERWRITE)
embedding = OpenAIDocumentEmbedder(model="text-embedding-ada-002", 
                                    batch_size=24,
                                    )

# Initialize pipeline
indexing_pipeline = Pipeline()

# Add components
indexing_pipeline.add_component("converter", html_to_document)
indexing_pipeline.add_component("cleaner", document_cleaner)
indexing_pipeline.add_component("splitter", document_splitter)
indexing_pipeline.add_component("embedder", embedding)
indexing_pipeline.add_component("writer", document_writer)

# Connect components to one another
indexing_pipeline.connect("converter", "cleaner")
indexing_pipeline.connect("cleaner.documents", "splitter.documents")
indexing_pipeline.connect("splitter.documents", "embedder.documents")
indexing_pipeline.connect("embedder.documents", "writer.documents")

# Execute pipeline
file_names = [str(f) for f in Path("./markdown_pages").rglob("*.md")]
indexing_pipeline.run({"converter": {"sources": file_names}})
```

## Strategy - wrap pipeline in a custom component

```python
from haystack import component

@component
class IndexingPipeline:
  """
  A component wrapping the indexing pipeline
  """
  @component.output_types(welcome_text=str, note=str)
  def run(self, name:str):
    indexing_pipeline.run({"converter": {"sources": file_names}})
    return {"welcome_text": "Indexing pipeline executed", 
            "note": "documents are indexed"}
```